In [1]:
## These notes happened chrnologically. so bottom is current moment.

current best test score 0.9203326182684942
That is with the entire kitchen sink thrown at it.
but with squarefoot not combined

current best test score 0.9212150618551485

train, test = add_price_comp_feature(train, test,'Neighborhood')
train, test = add_price_comp_feature(train, test,'GarageCars')
train, test = add_price_comp_feature(train, test,'BldgType')
train, test = add_price_comp_feature(train, test,'MSZoning')
train, test = add_price_comp_feature(train, test,'Condition1')

smaller_num_feat_test = [
    'OverallQual',
    'overall_score',
    'total_sf',
    'GrLivArea',
    'year_since_built',
    'LotArea',
    'GarageArea',
    'year_since_remod',
    'BsmtExposure',
    'KitchenQual'
    
]

current best test score: 0.9235239696528883

this is with the absolute kitchen sink including every the squarefoot combo stuff

current best test score: 0.9230459482712876

this is with the absolute kitchen sink including every the squarefoot combo stuff and now the parks informations

0.9204376273650542

has location information. same list above. now with all log_comp and comp

0.920675705756993
most up to date schools/parks. but only looked at closest school/park and not distacne to any given park. same short list of ten 

Big difference only did log comp and not normal comp. but i could remove the closest school/park log comp

0.9205881222491139 same as the above but now just normal comp. gonna switch back to log comps and remove the logcomp of school and park

0.9206757265923926 

log comp same as above but now removing the closetschool/park as talked about above.

0.9206784978665692
parks/schools as cats and both logcomp and comp

0.9208968850972118 without schools/parks

0.9195358468343512 just parks

0.9209695687705756 just schools

In [2]:
import pandas as pd
import numpy as np
import helper
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import SelectFromModel

In [3]:
colors = ["#FF0B04", "#F1BE48",
           "#B9975B", "#8B5B29",
           "#524727",
         ]
sns.set_palette(sns.color_palette(colors))

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [5]:
housing = pd.read_csv('Ames_Housing_Price_Data.csv', index_col=0,low_memory = False)


# train_, test_ = helper.data_processing_wrapper(housing,
#                                                num_to_cat_list = ['MSSubClass','MoSold'],
#                                              remove_PID = False
#                                         )
train, test = helper.data_processing_wrapper(housing,
                                               num_to_cat_list = ['MSSubClass','MoSold'],
                                             remove_PID = False
                                        )




In [6]:
parks = pd.read_csv('parksFeatures.csv',index_col = 0)
# Keeping only Park features that had the biggest result.
park_keep = [
    'PID',
    'closestPark'
]
parks = parks[park_keep]

In [7]:
# train = train_.merge(parks, how = 'left', left_on = 'PID', right_on = 'PID')
# test = test_.merge(parks, how = 'left', left_on = 'PID', right_on = 'PID')

# train = train.dropna(subset=['closestPark'])
# train = train.reset_index(drop=True)

# test = test.dropna(subset=['closestPark'])
# test = test.reset_index(drop=True)

In [8]:
schools = pd.read_csv('schoolFeatures.csv',index_col = 0)
school_keep = [
    'PID',
    'closestSchool'
]
schools = schools[school_keep]

In [9]:
train = train.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')
test = test.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')

train = train.dropna(subset=['closestSchool'])
train = train.reset_index(drop=True)

test = test.dropna(subset=['closestSchool'])
test = test.reset_index(drop=True)

In [10]:
cat_feats = train.select_dtypes(['object','bool']).columns.to_list()

In [11]:
num_cols = train.select_dtypes(['float64','int64']).columns.to_list()
num_cols.remove('SalePrice')

define added feature functions

In [12]:
def add_year_since_feature(df):
    df['year_since_built'] = df['YrSold']-df['YearBuilt']
    df['year_since_remod'] = df['YrSold']-df['YearRemodAdd']
    df['year_since_garage'] = df['YrSold']-df['GarageYrBlt']

    df.loc[df['year_since_built']<0,'year_since_built']=0
    df.loc[df['year_since_remod']<0,'year_since_remod']=0
    df.loc[df['year_since_garage']<0,'year_since_garage']=0
    return df

In [13]:
def add_combined_related_num_features(df):
    df['total_sf'] = df['GrLivArea'] + df['TotalBsmtSF']
    df['total_high_qual_finished_sf'] = (df['GrLivArea'] + df['TotalBsmtSF'] - # Adding total square foot.
                                         df['BsmtUnfSF'] - df['LowQualFinSF']) # Subtracting unfinished/low qual
    df['total_deck_sf'] = (df['WoodDeckSF'] + df['OpenPorchSF'] + df['EnclosedPorch'] + 
                           df['3SsnPorch'] + df['ScreenPorch'])
    df['total_full_bath'] = df['BsmtFullBath'] + df['FullBath']
    df['total_half_bath'] = df['BsmtHalfBath'] + df['HalfBath']
    return df

In [14]:
def add_score_feature(df):
    df['overall_score'] = df['OverallQual']*df['OverallCond']
    df['exter_score'] = df['ExterQual']*df['ExterCond']
    df['bsmt_score'] = df['BsmtQual']*df['BsmtCond']
    df['garage_score'] = df['GarageQual']*df['GarageCond']
    return df


In [15]:
def add_non_linear_transformed_features(df,cols):
    df_list = [df]
    for col in cols:
        df_new = pd.DataFrame()
        df_new[col+'_squared'] = df[col]**2
        df_new[col+'_cubed'] = df[col]**3
        df_new[col+'_square_root'] = df[col]**0.5
        df_list.append(df_new)
    df = pd.concat(df_list, axis=1)
    return df

In [16]:
# 
def add_price_comp_feature(train_, test_,comp_feature):
    temp = train.groupby(comp_feature).agg({'SalePrice':'median'})
    temp.columns = [comp_feature+'_comp']
    train_ = train_.merge(temp, how='left', on=comp_feature)
    test_ = test_.merge(temp, how='left', on=comp_feature)
    return train_, test_


In [17]:
def add_price_comp_log_feature(train_, test_,comp_feature):
    temp = train_.copy()
    temp['log_SalePrice'] = np.log(temp['SalePrice'])
    temp = temp.groupby(comp_feature).agg({'log_SalePrice':'median'})
    temp.columns = [comp_feature+'_log_comp']
    train_ = train_.merge(temp, how='left', on=comp_feature)
    test_ = test_.merge(temp, how='left', on=comp_feature)
    return train_, test_

In [18]:
def lasso_grid_cv(train_,test_,cat_feats_,
                  starting_alphas_=[0.0001, 0.0003, 0.0006, 0.001, 
                                    0.003, 0.006, 0.01, 0.03, 0.06, 0.1,
                                    0.3, 0.6, 1],
                  n_jobs_ = None,
                  cv_ = 5
                  
                 ):

    scaler = StandardScaler(with_mean=False)
    lasso = Lasso(max_iter = 50000, )

    X = train_.drop(['SalePrice','PID'],axis=1)
    transformer = ColumnTransformer([("Cat", 
                                      OneHotEncoder(handle_unknown = 'ignore'), 
                                      cat_feats_)], remainder='passthrough')
    X = transformer.fit_transform(X)
    X = scaler.fit_transform(X)
    y = np.log(train['SalePrice'])

    # Grid Search set up.

    alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                              0.3, 0.6, 1]

    tuned_parameters = [{'alpha': alphas}]
    print(f'Performing Grid Search with alphas of: {alphas}')
    clf = GridSearchCV(lasso, tuned_parameters, cv=cv_,n_jobs = n_jobs_)
    clf.fit(X, y)

    # best alpha with first draft. Now iterate for more precision with alphas.
    best_alpha = clf.best_params_['alpha']
    best_score = clf.best_score_
    print(f'Current best alpha: {best_alpha}')
    print(f'Current best CV R2: {best_score}')
    best_score_last = 1
    best_score_diff = abs(best_score-best_score_last)
    while best_score_diff > .001:
        best_score_last = best_score
        alphas_multiply = np.array([.3,.4,.5,.6,.7,.8,.9,1,
                            1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9])
        alphas = alphas_multiply*best_alpha
        tuned_parameters = [{'alpha': alphas}]
        print(f'Performing Grid Search with alphas of: {alphas}')
        clf = GridSearchCV(lasso, tuned_parameters, cv=5)
        clf.fit(X, y)
        best_alpha = clf.best_params_['alpha']
        best_score = clf.best_score_
        
        print(f'Current best alpha: {best_alpha}')
        print(f'Current best CV R2: {best_score}')        
        best_score_diff = abs(best_score-best_score_last)
    print('Modeling complete :)')
    return clf, transformer, scaler

In [19]:

# train, test = add_price_comp_feature(train, test,'Neighborhood')
# train, test = add_price_comp_feature(train, test,'GarageCars')
# train, test = add_price_comp_feature(train, test,'BldgType')
# train, test = add_price_comp_feature(train, test,'MSZoning')
# train, test = add_price_comp_feature(train, test,'Condition1')


train, test = add_price_comp_log_feature(train, test,'Neighborhood')
train, test = add_price_comp_log_feature(train, test,'GarageCars')
train, test = add_price_comp_log_feature(train, test,'BldgType')
train, test = add_price_comp_log_feature(train, test,'MSZoning')
train, test = add_price_comp_log_feature(train, test,'Condition1')



In [20]:
smaller_num_feat_test = [
    'OverallQual',
    'overall_score',
    'total_sf',
    'GrLivArea',
    'year_since_built',
    'LotArea',
    'GarageArea',
    'year_since_remod',
    'BsmtExposure',
    'KitchenQual'
]

In [21]:
train = add_year_since_feature(train)
train = add_score_feature(train)
train = add_combined_related_num_features(train)
train = add_non_linear_transformed_features(train,smaller_num_feat_test)

In [22]:
test = add_year_since_feature(test)
test = add_score_feature(test)
test = add_combined_related_num_features(test)
test = add_non_linear_transformed_features(test,smaller_num_feat_test)

In [23]:
# train, test = add_price_comp_feature(train, test,'overall_score')

In [24]:
num_cols = train.select_dtypes(['float64','int64']).columns.to_list()
num_cols.remove('SalePrice')
num_cols.remove('PID')

In [25]:
clf, transformer, scaler = lasso_grid_cv(train,test,cat_feats,n_jobs_ = -1)

Performing Grid Search with alphas of: [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1]
Current best alpha: 0.001
Current best CV R2: 0.9473110960763804
Performing Grid Search with alphas of: [0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009 0.001  0.0011 0.0012
 0.0013 0.0014 0.0015 0.0016 0.0017 0.0018 0.0019]
Current best alpha: 0.0013000000000000002
Current best CV R2: 0.9474134116268011
Modeling complete :)


In [26]:
clf.best_score_

0.9474134116268011

In [27]:
X_tst = test.drop(['SalePrice','PID'],axis=1)
X_tst = transformer.transform(X_tst)
X_tst = scaler.transform(X_tst)
y_tst = np.log(test['SalePrice'])

clf.score(X_tst,y_tst)

0.9209695687705756

In [28]:
columns_transformed = transformer.named_transformers_['Cat'].get_feature_names(input_features= cat_feats)
new_columns = list(columns_transformed)+num_cols

coef_df = pd.DataFrame({'features':new_columns,'coefs':clf.best_estimator_.coef_})
coef_df = coef_df[coef_df['coefs']!=0]
coef_df['coefs_abs'] = abs(coef_df['coefs'])
coef_df = coef_df.sort_values('coefs_abs',ascending=False).reset_index(drop=True)
coef_df


,features,coefs,coefs_abs
0,GrLivArea_square_root,6.517607e-02,6.517607e-02
1,year_since_built_square_root,-6.357817e-02,6.357817e-02
2,total_sf_square_root,6.320086e-02,6.320086e-02
3,overall_score_square_root,5.157190e-02,5.157190e-02
4,LotArea_square_root,3.556620e-02,3.556620e-02
5,total_high_qual_finished_sf,3.390390e-02,3.390390e-02
6,OverallQual_cubed,3.298275e-02,3.298275e-02
7,Neighborhood_log_comp,2.520868e-02,2.520868e-02
8,BsmtExposure_cubed,1.358697e-02,1.358697e-02
9,GarageCars,1.349737e-02,1.349737e-02


In [29]:
# GarageCars_log_comp
# Condition1_log_comp
# BldgType_log_comp
# MSZoning_log_comp
# Neighborhood_log_comp

Neighborhood_comp
MSZoning_log_comp
Condition1_log_comp
BldgType_comp
Condition1_comp
GarageCars_comp